In [1]:
import requests
import pandas as pd
import dotenv
import os
import datetime
import aiohttp
import asyncio
from tqdm.asyncio import tqdm
import psycopg2


In [2]:
dotenv.load_dotenv()

True

### Definições para requisição http

Definição das variáveis para requisição http à api do github

In [3]:
# Configurações
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")  # Substitua pelo seu token
REPO_OWNER = "CSSEGISandData"
REPO_NAME = "COVID-19"
API_URL = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/"
GITHUB_API_GRAPHQL_URL = "https://api.github.com/graphql"


HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}"
}

### Testando Limites de Requisição

Abaixo, bloco para testar quantas requisições faltam e quanto tempo falta para zerar o limite de requisições

In [4]:
response = requests.get("https://api.github.com/rate_limit", headers=HEADERS)

# Verifica o status da resposta
if response.status_code == 200:
    rate_limit = response.json()
    core = rate_limit['rate']
    limit = core['limit']
    remaining = core['remaining']
    reset_timestamp = core['reset']
    reset_time = datetime.datetime.fromtimestamp(reset_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    print(f"Limite de requisições por hora: {limit}")
    print(f"Requisições restantes: {remaining}")
    print(f"Limite será zerado em: {reset_time}")
else:
    print(f"Erro ao verificar rate limit: {response.status_code}")

Limite de requisições por hora: 5000
Requisições restantes: 5000
Limite será zerado em: 2025-02-22 22:55:21


In [5]:
query = """
{
  rateLimit {
    limit
    remaining
    resetAt
  }
}
"""

response = requests.post(GITHUB_API_GRAPHQL_URL, json={'query': query}, headers=HEADERS)

if response.status_code == 200:
    rate_limit = response.json()['data']['rateLimit']
    limit = rate_limit['limit']
    remaining = rate_limit['remaining']
    reset_timestamp = rate_limit['resetAt']
    reset_time = datetime.datetime.strptime(reset_timestamp, "%Y-%m-%dT%H:%M:%SZ")

    print(f"Limite de requisições por hora: {limit}")
    print(f"Requisições restantes: {remaining}")
    print(f"Limite será zerado em: {reset_time.strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print(f"Erro ao verificar rate limit: {response.status_code}")

Limite de requisições por hora: 5000
Requisições restantes: 5000
Limite será zerado em: 2025-02-23 02:55:23


### Listagem dos arquivos presentes no repositório

Bloco para listar todos os arquivos .csv presentes no repositório, da raiz às subpastas.

Foi utilizada a api graphql para conseguir obter a lista de arquivos com menos requisições, assim, economizando o limite de requisições por hora da api do github

In [6]:
# Query GraphQL para listar arquivos e subpastas
query = """
query ($caminho: String!) {
  repository(owner: "CSSEGISandData", name: "COVID-19") {
    object(expression: $caminho) {
      ... on Tree {
        entries {
          name
          type
          oid
        }
      }
    }
  }
}
"""

# Função recursiva para buscar arquivos CSV com caminho completo e URL de download
def listar_arquivos_recursivo(caminho="master:csse_covid_19_data"):
    arquivos = []
    variables = {"caminho": caminho}
    response = requests.post(GITHUB_API_GRAPHQL_URL, json={'query': query, 'variables': variables}, headers=HEADERS)
    
    if response.status_code == 200:
        dados = response.json()
        if 'data' in dados and dados['data']['repository']['object']:
            entradas = dados['data']['repository']['object']['entries']
            
            for item in entradas:
                # Se for subpasta, chamar recursivamente
                if item['type'] == 'tree':
                    subcaminho = caminho + "/" + item['name']
                    arquivos.extend(listar_arquivos_recursivo(subcaminho))  # Recurso recursivo
                # Se for CSV, adicionar à lista
                elif item['type'] == 'blob' and item['name'].endswith('.csv'):
                    caminho_sem_arquivo = caminho.replace("master:", "")
                    url_download = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/{caminho_sem_arquivo}/{item['name']}"
                    arquivos.append({
                        "caminho_completo": caminho_sem_arquivo,
                        "nome_arquivo": item['name'],
                        "url_download": url_download
                    })
        else:
            print("Nenhum dado encontrado para o caminho:", caminho)
    else:
        print("Erro na requisição:", response.status_code, response.text)
    
    return arquivos

# Executar a função e armazenar em DataFrame
lista_csvs = listar_arquivos_recursivo()
df_csvs = pd.DataFrame(lista_csvs)

print("Arquivos CSV encontrados:")
display(df_csvs)



Arquivos CSV encontrados:


,caminho_completo,nome_arquivo,url_download
0,csse_covid_19_data,UID_ISO_FIPS_LookUp_Table.csv,https://raw.githubusercontent.com/CSSEGISandDa...
1,csse_covid_19_data/csse_covid_19_daily_reports,01-01-2021.csv,https://raw.githubusercontent.com/CSSEGISandDa...
2,csse_covid_19_data/csse_covid_19_daily_reports,01-01-2022.csv,https://raw.githubusercontent.com/CSSEGISandDa...
3,csse_covid_19_data/csse_covid_19_daily_reports,01-01-2023.csv,https://raw.githubusercontent.com/CSSEGISandDa...
4,csse_covid_19_data/csse_covid_19_daily_reports,01-02-2021.csv,https://raw.githubusercontent.com/CSSEGISandDa...
...,...,...,...
2207,csse_covid_19_data/csse_covid_19_time_series,time_series_covid19_confirmed_US.csv,https://raw.githubusercontent.com/CSSEGISandDa...
2208,csse_covid_19_data/csse_covid_19_time_series,time_series_covid19_confirmed_global.csv,https://raw.githubusercontent.com/CSSEGISandDa...
2209,csse_covid_19_data/csse_covid_19_time_series,time_series_covid19_deaths_US.csv,https://raw.githubusercontent.com/CSSEGISandDa...
2210,csse_covid_19_data/csse_covid_19_time_series,time_series_covid19_deaths_global.csv,https://raw.githubusercontent.com/CSSEGISandDa...


In [7]:
# Função para baixar um arquivo CSV mantendo a estrutura de diretórios
async def baixar_arquivo(session, url_download, caminho_completo, nome_arquivo):
    # Cria o caminho completo do diretório de destino
    pasta_destino = os.path.join("files", caminho_completo)
    os.makedirs(pasta_destino, exist_ok=True)
    caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
    
    # Se o arquivo já existe, pula o download
    if os.path.exists(caminho_arquivo):
        print(f"{nome_arquivo} já existe, pulando download.")
        return

    # Faz o download do arquivo
    async with session.get(url_download) as response:
        if response.status == 200:
            conteudo = await response.read()
            with open(caminho_arquivo, 'wb') as f:
                f.write(conteudo)
        else:
            print(f"Falha ao baixar {nome_arquivo}: {response.status}")

# Função principal para baixar todos os CSVs com estrutura de pastas
async def baixar_csvs_assincrono(arquivos):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for arquivo in arquivos:
            url_download = arquivo['url_download']
            caminho_completo = arquivo['caminho_completo']
            nome_arquivo = arquivo['nome_arquivo']
            tasks.append(baixar_arquivo(session, url_download, caminho_completo, nome_arquivo))
        
        # Executa as tasks com tqdm para mostrar o progresso
        for _ in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Baixando arquivos"):
            await _
            
# Executar a função para baixar os CSVs
await baixar_csvs_assincrono(df_csvs.to_dict(orient='records'))

Baixando arquivos:   0%|          | 0/2212 [00:00<?, ?it/s]

02-25-2020.csv já existe, pulando download.
06-12-2021.csv já existe, pulando download.
11-08-2022.csv já existe, pulando download.
04-18-2020.csv já existe, pulando download.
09-14-2021.csv já existe, pulando download.
02-25-2021.csv já existe, pulando download.
06-12-2022.csv já existe, pulando download.
11-09-2020.csv já existe, pulando download.
04-18-2021.csv já existe, pulando download.
09-14-2022.csv já existe, pulando download.
02-25-2022.csv já existe, pulando download.
06-13-2020.csv já existe, pulando download.
11-09-2021.csv já existe, pulando download.
04-18-2022.csv já existe, pulando download.
09-15-2020.csv já existe, pulando download.
02-25-2023.csv já existe, pulando download.
06-13-2021.csv já existe, pulando download.
11-09-2022.csv já existe, pulando download.
04-19-2020.csv já existe, pulando download.
09-15-2021.csv já existe, pulando download.
02-26-2020.csv já existe, pulando download.
06-13-2022.csv já existe, pulando download.
11-10-2020.csv já existe, puland

Baixando arquivos: 100%|██████████| 2212/2212 [00:00<00:00, 6588.65it/s]

02-03-2022.csv já existe, pulando download.
07-13-2022.csv já existe, pulando download.
12-10-2020.csv já existe, pulando download.
05-18-2021.csv já existe, pulando download.
08-19-2022.csv já existe, pulando download.
02-03-2023.csv já existe, pulando download.
07-14-2020.csv já existe, pulando download.
12-10-2021.csv já existe, pulando download.
05-18-2022.csv já existe, pulando download.
08-20-2020.csv já existe, pulando download.
02-04-2021.csv já existe, pulando download.
07-14-2021.csv já existe, pulando download.
12-10-2022.csv já existe, pulando download.
05-19-2020.csv já existe, pulando download.
08-20-2021.csv já existe, pulando download.
02-04-2022.csv já existe, pulando download.
07-14-2022.csv já existe, pulando download.
12-11-2020.csv já existe, pulando download.
05-19-2021.csv já existe, pulando download.
08-20-2022.csv já existe, pulando download.
02-04-2023.csv já existe, pulando download.
07-15-2020.csv já existe, pulando download.
12-11-2021.csv já existe, puland

In [8]:
print(os.getcwd())

s:\git\otg\analista-bi


In [9]:
# Diretório onde os arquivos CSV estão localizados
diretorio_csv = os.path.join(os.getcwd(),'files','csse_covid_19_data','csse_covid_19_daily_reports')

def processar_csvs(diretorio_csv):
    """
    Processa os arquivos CSV, adicionando a coluna 'date' e concatenando os dados.
    """
    dfs = []
    
    # Usando tqdm para mostrar o progresso ao ler os arquivos CSV
    for arquivo in tqdm(os.listdir(diretorio_csv), desc="Processando arquivos CSV", unit="arquivo"):
        if arquivo.endswith('.csv'):
            # Remover a extensão .csv e extrair a data
            arquivo_sem_extensao = arquivo.replace('.csv', '')
            data = arquivo_sem_extensao.split('-')
            data = f"{data[2]}-{data[0]}-{data[1]}"  # Formata como 'yyyy-mm-dd'
            
            # Carregar o CSV em um DataFrame
            df = pd.read_csv(os.path.join(diretorio_csv, arquivo))
            
            # Adicionar a coluna 'date'
            df['date'] = data
            
            # Adicionar à lista
            dfs.append(df)
    
    # Concatenar todos os DataFrames em um único
    df_final = pd.concat(dfs, ignore_index=True)
    return df_final

df_final = processar_csvs(diretorio_csv)

print("Arquivos concatenados com sucesso, incluindo a coluna de data!")

Processando arquivos CSV: 100%|██████████| 1143/1143 [00:40<00:00, 28.47arquivo/s]


Arquivos concatenados com sucesso, incluindo a coluna de data!


In [ ]:
print(os.path.join(os.getcwd(),'files','final','daily_reports_concat.csv'))

dir = os.path.join(os.getcwd(),'files','final')
# Salvar o DataFrame final em um novo arquivo CSV
if not os.path.exists(dir):
    os.mkdir(dir)
    
df_final.to_csv(os.path.join(dir,'daily_reports_concat.csv'), index=False)

In [12]:
print(df_final.columns)
print(df_final.head())

Index(['fips', 'admin2', 'province_state', 'country_region', 'last_update',
       'lat', 'long', 'confirmed', 'deaths', 'recovered', 'active',
       'combined_key', 'incident_rate', 'case_fatality_ratio', 'date',
       'province_state_extra', 'country_region_extra', 'last_update_extra',
       'latitude', 'longitude', 'incidence_rate', 'case_fatality_ratio_extra'],
      dtype='object')
   fips admin2 province_state country_region          last_update       lat  \
0   NaN    NaN            NaN    Afghanistan  2021-01-02 05:22:33  33.93911   
1   NaN    NaN            NaN        Albania  2021-01-02 05:22:33  41.15330   
2   NaN    NaN            NaN        Algeria  2021-01-02 05:22:33  28.03390   
3   NaN    NaN            NaN        Andorra  2021-01-02 05:22:33  42.50630   
4   NaN    NaN            NaN         Angola  2021-01-02 05:22:33 -11.20270   

        long  confirmed  deaths  recovered  ...  incident_rate  \
0  67.709953    52513.0  2201.0    41727.0  ...     134.896578   


In [ ]:
# Renomeando as colunas para usar underscores
    df_final.rename(columns={
        'FIPS': 'fips',
        'Admin2': 'admin2',
        'Province_State': 'province_state',
        'Country_Region': 'country_region',
        'Last_Update': 'last_update',
        'Lat': 'lat',
        'Long_': 'long',
        'Confirmed': 'confirmed',
        'Deaths': 'deaths',
        'Recovered': 'recovered',
        'Active': 'active',
        'Combined_Key': 'combined_key',
        'Incident_Rate': 'incident_rate',
        'Case_Fatality_Ratio': 'case_fatality_ratio',
        'Province/State': 'province_state_extra',
        'Country/Region': 'country_region_extra',
        'Last Update': 'last_update_extra',
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'Incidence_Rate': 'incidence_rate',
        'Case-Fatality_Ratio': 'case_fatality_ratio_extra'
    }, inplace=True)
    
    # Selecionando as colunas na ordem correta
    colunas = [
        'fips', 'admin2', 'province_state', 'country_region', 'last_update', 
        'lat', 'long', 'confirmed', 'deaths', 'recovered', 'active', 
        'combined_key', 'incident_rate', 'case_fatality_ratio', 'date', 
        'province_state_extra', 'country_region_extra', 'last_update_extra', 
        'latitude', 'longitude', 'incidence_rate', 'case_fatality_ratio_extra'
    ]
    
    # Garantindo que apenas essas colunas estejam presentes no DataFrame
    df = df[colunas]

In [11]:
def conectar_postgresql(host, database, user, password, port="5432"):
    """
    Estabelece a conexão com o banco de dados PostgreSQL.
    """
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    return conn

def criar_tabela(cur):
    """
    Cria a tabela no PostgreSQL se ela não existir, incluindo todas as colunas do DataFrame.
    """
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS daily_reports (
        fips TEXT,
        admin2 TEXT,
        province_state TEXT,
        country_region TEXT,
        last_update TEXT,
        lat DOUBLE PRECISION,
        long DOUBLE PRECISION,
        confirmed INTEGER,
        deaths INTEGER,
        recovered INTEGER,
        active INTEGER,
        combined_key TEXT,
        incident_rate DOUBLE PRECISION,
        case_fatality_ratio DOUBLE PRECISION,
        date DATE,
        province_state_extra TEXT,
        country_region_extra TEXT,
        last_update_extra TEXT,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        incidence_rate DOUBLE PRECISION,
        case_fatality_ratio_extra DOUBLE PRECISION
    );
    '''
    cur.execute(create_table_query)
    
def inserir_dados_postgresql(cur, conn, df):
    """
    Insere os dados do DataFrame no banco de dados PostgreSQL.
    """    
    insert_query = '''
    INSERT INTO daily_reports (
        fips, admin2, province_state, country_region, last_update, lat, long, 
        confirmed, deaths, recovered, active, combined_key, incident_rate, 
        case_fatality_ratio, date, province_state_extra, country_region_extra, 
        last_update_extra, latitude, longitude, incidence_rate, case_fatality_ratio_extra
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    '''
    
    # Usando tqdm para mostrar o progresso ao inserir os dados
    with tqdm(total=len(df), desc="Inserindo dados no PostgreSQL", unit="linha") as pbar:
        for idx, row in enumerate(df.itertuples(index=False, name=None)):
            cur.execute(insert_query, row)
            
            # Fazer commit a cada 1000 registros para melhorar desempenho
            if (idx + 1) % 1000 == 0:
                conn.commit()
            
            pbar.update(1)
    
    # Commit final para os registros restantes
    conn.commit()

        

dotenv.load_dotenv()

# Definir as credenciais de conexão
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

# Conectar ao banco de dados PostgreSQL
conn = conectar_postgresql(host, database, user, password)
cur = conn.cursor()

# Criar a tabela no banco de dados
criar_tabela(cur)

# Inserir os dados no banco de dados
inserir_dados_postgresql(cur, conn, df_final)

# Commit as changes and close the connection
conn.commit()

# Fechar o cursor e a conexão
cur.close()
conn.close()
print("Dados inseridos com sucesso no PostgreSQL!")


Inserindo dados no PostgreSQL:   0%|          | 0/4287473 [00:00<?, ?linha/s]


TypeError: not all arguments converted during string formatting